# MNIST

In [1]:
import h5py as h5
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
import numpy as np

In [2]:
data = h5.File("../Lecture_07/Files/mnist.h5")
X_train = data['train']['inputs'][()].astype(np.float32)/255
X_test = data['test']['inputs'][()].astype(np.float32)/255
y_train = data['train']['targets'][()].astype(np.long)
y_test = data['test']['targets'][()].astype(np.long)

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, Dataset, DataLoader, sampler
import torch.nn.functional as F

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.softmax(x, dim=1)

model = Net()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [5]:
X_train = np.array([i.reshape(1, 28, 28) for i in X_train])
X_test = np.array([i.reshape(1, 28, 28) for i in X_test])
train_set = torch.utils.data.TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_set = torch.utils.data.TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
train_loader = DataLoader(dataset=train_set, sampler=sampler.RandomSampler(train_set), batch_size=100)
test_loader = DataLoader(dataset=test_set, sampler=sampler.RandomSampler(test_set), batch_size=100)

def train(model, train_loader, optimizer):
    model.train()
    for epoch in range(10):
        for data, target in train_loader:
            optimizer.zero_grad()
            prediction = model(data)
            loss = F.cross_entropy(prediction, target)
            loss.backward()
            optimizer.step()

def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
train(model, train_loader, optimizer)
test(model, test_loader)


Test set: Average loss: -0.9766, Accuracy: 9773/10000 (98%)

